In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import joblib

# قراءة البيانات
data = pd.read_csv('Feedback_answers.csv')

# تحويل الفئات النصية إلى أرقام
label_encoder = LabelEncoder()

# تحويل SleepQuality باستخدام LabelEncoder
data['SleepQuality'] = label_encoder.fit_transform(data['SleepQuality'])

# تحويل باقي الأعمدة النصية باستخدام pd.get_dummies
categorical_cols = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7']
data = pd.get_dummies(data, columns=categorical_cols)

# تقسيم البيانات إلى مدخلات ومخرجات
X = data.drop(['SleepQuality', 'UserID', 'Date'], axis=1)
y = data['SleepQuality']

# تقسيم البيانات إلى مجموعة تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تدريب نموذج RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# التنبؤ على مجموعة الاختبار
y_pred = model.predict(X_test)

# الحصول على الفئات الفعلية
actual_classes = label_encoder.classes_[y_test.unique()]

# تقييم النموذج
print(classification_report(y_test, y_pred, target_names=actual_classes))

# تعريف مجموعة المعلمات للتجربة
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# إنشاء نموذج GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')

# تدريب النموذج
grid_search.fit(X_train, y_train)

# الحصول على أفضل المعلمات
best_params = grid_search.best_params_
print("أفضل المعلمات: ", best_params)

# استخدام أفضل معلمات لتدريب النموذج النهائي
best_model = grid_search.best_estimator_

# التنبؤ وتقييم النموذج
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best, target_names=actual_classes))

# تحضير المدخلات الجديدة
answers = ['No', 'Yes', 'Yes', 'Moderately noisy', 'Cool', 'Yes', 'No']

# تحويل المدخلات إلى نفس التنسيق المستخدم في التدريب
input_data = {
    'Q1': [answers[0]],
    'Q2': [answers[1]],
    'Q3': [answers[2]],
    'Q4': [answers[3]],
    'Q5': [answers[4]],
    'Q6': [answers[5]],
    'Q7': [answers[6]]
}

input_df = pd.DataFrame(input_data)
input_df = pd.get_dummies(input_df)

# ضمان أن تكون الأعمدة في المدخلات الجديدة تتطابق مع الأعمدة المستخدمة في التدريب
missing_cols = set(X.columns) - set(input_df.columns)
for col in missing_cols:
    input_df[col] = 0
input_df = input_df[X.columns]

# تنفيذ التنبؤ
predicted_quality_encoded = best_model.predict(input_df)

# تحويل التنبؤ إلى التصنيف الأصلي
predicted_quality = label_encoder.inverse_transform(predicted_quality_encoded)

print(f"Predicted Sleep Quality: {predicted_quality[0]}")

# نصائح بناءً على الإجابات
explanations_recommendations = {
    "Q1": ("Stress and anxiety can interfere with sleep.", "Practice relaxation techniques before bed."),
    "Q2": ("Nicotine is a stimulant that can disrupt sleep.", "Avoid nicotine for at least 4-6 hours before bedtime."),
    "Q3": ("Blue light from devices can suppress melatonin production.", "Avoid devices 1-2 hours before bedtime."),
    "Q4": ("Noise can disrupt sleep.", "Create a quiet sleep environment."),
    "Q5": ("Extreme temperatures can interfere with sleep.", "Maintain a warm bedroom temperature."),
    "Q6": ("Caffeine is a stimulant that can interfere with sleep.", "Avoid caffeine for at least 4-6 hours before bedtime."),
    "Q7": ("Eating close to bedtime can disrupt sleep.", "Finish eating at least 2-3 hours before bedtime."),
}

reasons = []
recommendations = []

if predicted_quality[0] in ['Poor', 'Average']:
    print("Your sleep quality was not optimal. It might be due to one or more of the following reasons:")
    for i, answer in enumerate(answers, start=1):
        if answer in ['Yes','Yes', ['Moderately noisy', 'Noisy'], ['Cool', 'Hot'],'Yes','Yes']:
            q_key = f"Q{i}"
            explanation, recommendation = explanations_recommendations[q_key]
            reasons.append(explanation)
            recommendations.append(recommendation)

# عرض الأسباب
print("Your sleep was not good, it could be due to the following reasons:")
for reason in reasons:
    print(f"- {reason}")

# عرض النصائح
print("\nHere are some tips to consider for better sleep:")
for recommendation in recommendations:
    print(f"- {recommendation}")

# حفظ النموذج المدرب
joblib.dump(best_model, 'best_model.pkl')

# حفظ الـ LabelEncoder
joblib.dump(label_encoder, 'label_encoder.pkl')


              precision    recall  f1-score   support

     Average       0.97      0.94      0.95        32
        Poor       1.00      0.90      0.95        10
        Good       0.90      1.00      0.95        18

    accuracy                           0.95        60
   macro avg       0.96      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60



C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


أفضل المعلمات:  {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 100}
              precision    recall  f1-score   support

     Average       0.79      0.84      0.82        32
        Poor       1.00      0.30      0.46        10
        Good       0.78      1.00      0.88        18

    accuracy                           0.80        60
   macro avg       0.86      0.71      0.72        60
weighted avg       0.82      0.80      0.78        60

Predicted Sleep Quality: Average
Your sleep quality was not optimal. It might be due to one or more of the following reasons:
Your sleep was not good, it could be due to the following reasons:
- Nicotine is a stimulant that can disrupt sleep.
- Blue light from devices can suppress melatonin production.
- Caffeine is a stimulant that can interfere with sleep.

Here are some tips to consider for better sleep:
- Avoid nicotine for at least 4-6 hours before bedtime.
- Avoid devices 1-2 hours before bedtime.
- Avoid caffeine for at least 

['label_encoder.pkl']

In [17]:
import pandas as pd
import joblib

# تحميل النموذج المدرب و LabelEncoder
best_model = joblib.load('best_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# تحضير المدخلات الجديدة
answers = ['No', 'Yes', 'Yes', 'Moderately noisy', 'Cool', 'Yes', 'No']
# answers = ['No', 'No', 'No', 'Quiet', 'Cool', 'No', 'No']
# answers =   ['No', 'Yes', 'Yes', 'Moderately noisy', 'Cool', 'Yes', 'No']
# answers = ['No', 'No', 'No', 'Quiet', 'Hot', 'Yes', 'No']
# answers = answers =['No', 'No', 'No', 'Quiet', 'Warm', 'No', 'Yes']
# answers =   ['No', 'Yes', 'Yes', 'Moderately noisy', 'Hot', 'No', 'Yes']
# تحويل المدخلات إلى نفس التنسيق المستخدم في التدريب
input_data = {
    'Q1': [answers[0]],
    'Q2': [answers[1]],
    'Q3': [answers[2]],
    'Q4': [answers[3]],
    'Q5': [answers[4]],
    'Q6': [answers[5]],
    'Q7': [answers[6]]
}

input_df = pd.DataFrame(input_data)
input_df = pd.get_dummies(input_df)

# ضمان أن تكون الأعمدة في المدخلات الجديدة تتطابق مع الأعمدة المستخدمة في التدريب
# وذلك بإضافة أي أعمدة مفقودة وتعيينها إلى 0
missing_cols = set(best_model.feature_names_in_) - set(input_df.columns)
for col in missing_cols:
    input_df[col] = 0
input_df = input_df[best_model.feature_names_in_]

# تنفيذ التنبؤ
predicted_quality_encoded = best_model.predict(input_df)

# تحويل التنبؤ إلى التصنيف الأصلي
predicted_quality = label_encoder.inverse_transform(predicted_quality_encoded)

print(f"Predicted Sleep Quality: {predicted_quality[0]}")

# نصائح بناءً على الإجابات
explanations_recommendations = {
    "Q1": ("Stress and anxiety can interfere with sleep.", "Practice relaxation techniques before bed."),
    "Q2": ("Nicotine is a stimulant that can disrupt sleep.", "Avoid nicotine for at least 4-6 hours before bedtime."),
    "Q3": ("Blue light from devices can suppress melatonin production.", "Avoid devices 1-2 hours before bedtime."),
    "Q4": ("Noise can disrupt sleep.", "Create a quiet sleep environment."),
    "Q5": ("Extreme temperatures can interfere with sleep.", "Maintain a warm bedroom temperature."),
    "Q6": ("Caffeine is a stimulant that can interfere with sleep.", "Avoid caffeine for at least 4-6 hours before bedtime."),
    "Q7": ("Eating close to bedtime can disrupt sleep.", "Finish eating at least 2-3 hours before bedtime."),
}

reasons = []
recommendations = []

if predicted_quality[0] in ['Poor', 'Average']:
    for i, answer in enumerate(answers, start=1):
        if answer in ['Yes','Yes', 'Moderately noisy', 'Noisy', 'Cool', 'Hot','Yes','Yes',]:
        # if answer in['Yes','Moderately noisy', 'Noisy', 'Cool', 'Hot', 'Yes']:
            q_key = f"Q{i}"
            explanation, recommendation = explanations_recommendations[q_key]
            reasons.append(explanation)
            recommendations.append(recommendation)

if predicted_quality[0] in ['Poor', 'Average']:
    print("Your sleep was not good, it could be due to the following reasons:")
    for reason in reasons:
         print(f"- {reason}")
    print("\nHere are some tips to consider for better sleep:")
    for recommendation in recommendations:
         print(f"- {recommendation}")


Predicted Sleep Quality: Average
Your sleep was not good, it could be due to the following reasons:
- Nicotine is a stimulant that can disrupt sleep.
- Blue light from devices can suppress melatonin production.
- Noise can disrupt sleep.
- Extreme temperatures can interfere with sleep.
- Caffeine is a stimulant that can interfere with sleep.

Here are some tips to consider for better sleep:
- Avoid nicotine for at least 4-6 hours before bedtime.
- Avoid devices 1-2 hours before bedtime.
- Create a quiet sleep environment.
- Maintain a warm bedroom temperature.
- Avoid caffeine for at least 4-6 hours before bedtime.


In [ ]:
Predicted Sleep Quality: Average
Your sleep was not good, it could be due to the following reasons:
- Nicotine is a stimulant that can disrupt sleep.
- Blue light from devices can suppress melatonin production.
- Noise can disrupt sleep.
- Extreme temperatures can interfere with sleep.
- Caffeine is a stimulant that can interfere with sleep.

Here are some tips to consider for better sleep:
- Avoid nicotine for at least 4-6 hours before bedtime.
- Avoid devices 1-2 hours before bedtime.
- Create a quiet sleep environment.
- Maintain a warm bedroom temperature.
- Avoid caffeine for at least 4-6 hours before bedtime.